<center>
    <img src="http://sct.inf.utfsm.cl/wp-content/uploads/2020/04/logo_di.png" style="width:60%">
    <h1> INF285 - Computación Científica </h1>
    <h2> Laboratorio 3</h2>
    <h2> [S]cientific [C]omputing [T]eam </a> </h2>
    <h2> 2025-1</h2>
</center>

In [ ]:
!pip install pillow

In [ ]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import toeplitz

# Laboratorio 3: Modelado de desenfoque en imágenes mediante operadores lineales
# Funciones

Para la implementación de las preguntas, considere que **solo** tiene a su disposición las siguientes funciones:
* **np.linalg.svd(A, full_matrices=False)**: Calcula la descomposición en valores singulares (SVD) de la matriz **A**. Devuelve tres matrices: **U**, **Σ** (valores singulares) y **$V^\top$** tales que $A = U \Sigma V^\top$.
Por ejemplo `A = np.array([[1, 2], [3, 4], [5, 6]])`
le aplicamos la descomposicion svd `U, S, VT = np.linalg.svd(A, full_matrices=False)`
los output correspondientes serian:
    ```python
    U: [[-0.2298477   0.88346102]
        [-0.52474482  0.24078249]
        [-0.81964194 -0.40189603]]
    S: [9.52551809 0.51430058]
    VT: [[-0.61962948 -0.78489445]
         [-0.78489445  0.61962948]]

* **np.argsort(a)**: Devuelve los índices que ordenarían un array. Si el array es unidimensional, los índices corresponden al orden ascendente de los elementos. Para arrays multidimensionales, se aplica el ordenamiento aplanando el array primero.
`np.argsort([3, 1, 2])` devolvería `[1, 2, 0]`, ya que el índice del menor elemento osea el (1) es la posicion 1, seguido por el índice del siguiente menor elemento (2), que esta en la posicion 2, y finalmente el índice del mayor elemento (3), que esta en la posicion 0.
* **np.ravel(a)**: Devuelve una vista aplanada del array de entrada **a**. Si el array es multidimensional, se convierte en un array unidimensional sin copiar los datos siempre que sea posible.Por ejemplo, si se construye la matriz `a = np.array([[1,2],[3,4]])`, y se le aplica `np.ravel(a)` se obtiene `[1 2 3 4]`.

* **np.zeros_like(a)**: Devuelve un array de ceros con la misma forma y tipo que el array de entrada **a**. Esto es útil para inicializar matrices o arrays con valores nulos mientras se conserva la estructura del array original.  
Por ejemplo, si se construye la matriz `a = np.array([[1, 2], [3, 4]])`, y se le aplica `np.zeros_like(a)` se obtiene `array([[0, 0],[0, 0]])`

* **np.linalg.norm(x, ord='fro')**: Calcula la norma de Frobenius de una matriz **x**, que es la raíz cuadrada de la suma de los cuadrados de sus elementos. Es útil para medir la magnitud de una matriz o calcular errores relativos entre matrices.  
Por ejemplo, si se construye la matriz `x = np.array([[1, 2], [3, 4]])`, y se le aplica `np.linalg.norm(x, ord='fro')`, se obtiene el resultado `5.477225575051661`, ya que:  
$\sqrt{1^2 + 2^2 + 3^2 + 4^2} = \sqrt{30} \approx 5.477$.

* **np.diag(a)**: Esta función crea una matriz diagonal a partir de un vector plano (unidimensional). Los elementos del vector se colocan en la diagonal principal de una nueva matriz cuadrada, y el resto de los elementos se rellenan con ceros. Por ejemplo, si `a = np.array([1, 2, 3])`, entonces `np.diag(a)` produce la matriz:
    ```python
        [[1,0,0]
         [0,2,0]
         [0,0,3]]

In [ ]:
# Construcción de operadores de 'desenfoque'
def blur_operator(m, n, bl=1):
    """
    Genera los operadores de convolución 1-D empleados en el paper.

    inputs:
    ----------
    m, n : int
        Dimensiones de la imagen (filas = m, columnas = n).
    bl : {1, 2}
        1  → blur simétrico  (σ_r = σ_c = 0.01)
        2  → blur asimétrico (σ_r = 0.02, σ_c = 0.01)
    
    outputs:
    --------
    Ac : ndarray (m, m)
        Matriz Toeplitz que difumina horizontalmente (columnas).
    Ar : ndarray (n, n)
        Matriz Toeplitz que difumina verticalmente (filas).
    """

    # --- 1. Parámetros ----------------------------------------------------
    sigmac = 0.01
    sigmar = 0.02 if bl == 2 else sigmac
    # --- 2. Nucleos 1-D ---------------------------------------------------
    c_pos = np.linspace(0, 1, m)          # coordenadas normalizadas
    r_pos = np.linspace(0, 1, n)

    c = np.exp(-0.5 * (c_pos / sigmac) ** 2)
    r = np.exp(-0.5 * (r_pos / sigmar) ** 2)

    # umbral 1e-4 
    c[c < 1e-4] = 0.0
    r[r < 1e-4] = 0.0

    # normalización: división por (2·sum − valor central)
    c /= (2 * c.sum() - c[0])
    r /= (2 * r.sum() - r[0])

    # --- 3. Matrices Toeplitz --------------------------------------------
    Ac = toeplitz(c)          # (m × m)
    Ar = toeplitz(r)          # (n × n)
    return Ac, Ar



# Introducción al Laboratorio 3

En este laboratorio, exploraremos el modelado y la restauración de imágenes afectadas por desenfoque y ruido, utilizando herramientas matemáticas avanzadas como la descomposición en valores singulares (SVD). El objetivo principal es comprender cómo los operadores lineales, como las matrices de convolución, pueden simular el desenfoque en imágenes y cómo estas mismas herramientas pueden ser utilizadas para restaurar imágenes degradadas.

El proceso de desenfoque se modela mediante matrices Toeplitz que representan convoluciones gaussianas en las direcciones horizontal y vertical. Estas matrices, combinadas con ruido gaussiano, generan una imagen degradada que será el punto de partida para los algoritmos de restauración.

A lo largo del laboratorio, se implementarán técnicas basadas en SVD truncado para reconstruir la imagen original a partir de la imagen desenfocada y ruidosa. Además, se analizará el impacto de los valores singulares en la calidad de la reconstrucción, seleccionando los valores más relevantes para minimizar el error relativo entre la imagen original y la reconstruida.

# Proceso de desenfoque y degradación de la imagen 

Utilizamos un desenfoque Gaussiano **asimétrico** para lograr desenfocar la imagen. **En este caso se hará un desenfoque solo con $A_r$**, es decir, la imagen perturbada se obtendrá de la siguiente forma:
$$
    B_{\text{data}} = \underbrace{X_{\text{true}}\,A_r^\top}_{\displaystyle{B_{\text{true}}}} + \Xi
$$
donde $\Xi$ es el ruido agregado.
Para la caracterización del ruido $\Xi$  se utilizará el SNR (_signal-to-noise-ratio_), el cual se define de la siguiente forma,
$$
SNR = 10 \cdot \log_{10} \left( \frac{ \| B_{\text{true}} \|_F^2 }{ \| \Xi \|_F^2 } \right)
$$
la cual se especificará más adelante.

**NOTAR que este desenfoque propuesto es una versión reducida del presentado en el contexto del laboratorio, por lo cual debe tomar ventaja de esta simplificación significativa.**

A continuación se presenta el **forward problem**, es decir la construcción de una imagen degradada a partir de una imagen detallada.
Este procedimiento ya fue incluido en el contexto también.

In [ ]:
# Añadiendo ruido y desenfoque a la imagen
# -------------------------------
# Paso 1: Cargar imagen original y convertir a escala de grises
img = Image.open("img.jpg").convert("L")
img = img.resize((512, 512))
X_true = np.array(img) / 255.0  # Imagen original normalizada

# -------------------------------
# Creando las matrices de desenfoque gaussiano simétrico
_, Ar = blur_operator(512, 512, bl=1)
# -------------------------------
# Aplicando el desenfoque gaussiano a la imagen original
B_true = X_true @ Ar.T
# Añadiendo ruido gaussiano a la imagen desenfocada
E = np.load("ruido.npy")
B_data = B_true + E
snr_db = 10*np.log10(np.power(np.linalg.norm(B_true,'fro')/np.linalg.norm(E),2.))
# -------------------------------
# Visualización
plt.figure(figsize=(8, 5))
plt.subplot(1, 3, 1)
plt.imshow(X_true, cmap='gray')
plt.title("Imagen original(X_true)")
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(B_true, cmap='gray')
plt.title("Desenfoque gaussiano(B_true)")
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(B_data, cmap='gray')
plt.title(f"Con ruido SNR = {snr_db:.1f}dB (B_data)")
plt.axis('off')

plt.tight_layout()
plt.show()

A modo referencial, se recuerda que la relación entre la imagen degradada ($B_{\text{data}}$) y la imagen original ($X_{\text{true}}$) satisface la siguiente ecuación,
$$
    B_{\text{data}} = X_{\text{true}}\,A_r^{\top}+ \Xi
$$

## Proceso de truncamiento para limpiar el ruido y el desenfoque de la imagen $B_{\text{data}}$

Sea $A_r$ la matriz de convolución por filas, matriz transpuesta multiplicada por la derecha a la imagen.

Construyendo la factorización SVD de $A_r$ obtenemos:

$$
A_r = U_r \Sigma_r V_r^\top 
$$

Para obtener la reconstrucción $X_{\text{recon}}$ de la imagen original $X_{\text{true}}$, se necesita construir $(A_{r,\text{trunc}}^{\top})^{-1}$, la cual se obtendrá por medio de la SVD anterior.
Es decir, definimos la reconstrucción de la siguiente forma
$$
X_{\text{recon}} = B_{\text{data}}\,(A_{r,\text{trunc}}^{\top})^{-1}
$$
Utilizando la SVD de $A_r$, podemos obtener $(A_{r,\text{trunc}}^{\top})^{-1}$ de la siguiente forma,
$$
X_{\text{recon}} = B_{\text{data}}\,U_r\,\widehat{\Sigma}_r\,V_r^{\top}\,.
$$
donde,
$$
\left(\widehat{\Sigma}_r\right)_{i,j} =
\begin{cases}
    \frac{1}{(\sigma_r)_i} &  \text{ si $i \leq k$ e $i=j$}, \\
    0 & \text{en todo otro caso},
\end{cases}
$$
donde $i,j\in \{1,2,3,\dots,n\}$, es decir se indexan desde $1$.
**Esto significa que solo seleccionaremos los $k$ valores singulares $\sigma_r$ mayores para poder truncar el operador.**
Así, $\widehat{\Sigma}_r$ solo conserva la información de los valores singulares significativos, eliminando el efecto de los valores singulares pequeños asociados principalmente al ruido.
Ahora, por conveniencia, se define la siguiente matriz,
$$
\widehat{B} = B_{\text{data}}\,U_r
$$
Así, la reconstrucción queda:
$$
X_{\text{recon}} = \left(\widehat{B}\,\widehat{\Sigma}_r\, \right)\,V_r^{\top}
$$


### Pregunta 1.1 (20 pts)

En este pregunta se debe construir y retornar la matriz $\widehat{\Sigma}_r$ definida anteriormente.
Se recibe como parámetros un vector con los valores singulares de $A_r$ y $k$.

In [ ]:
def matriz_Sigma_hat(sigma,k):
    """
    input:
    sigma : array(n)
        Vector sigma (contiene los valores sigulares).      
    k : int
        Número de valores singulares a truncar.             
    output:
    Sigma_hat : ndarray (n, n)
        Devuelve una matriz con el recíproco de los k valores singulares mayores.
    """
    #--- acá va su código aquí ----
    
    #------------------------------ 
    return Sigma_hat

### Pregunta 1.2 (40 pts)
Implemente la restauración de la imagen $B_{\text{data}}$ usando el procedimento explicado anteriormente, cuando solo se le aplica un desenfoque por filas.

In [ ]:
def restaurar_imagen(B_data,Ar,k):
    """
    input:  
    B_data : ndarray (m, n)
        Imagen borrosa y ruidosa (con ruido gaussiano).
    Ar : ndarray (n, n)
        Matriz Toeplitz que difumina verticalmente (filas).
    k : int
        Número de valores singulares a truncar.
    output:
    X_recon : ndarray (m, n)
        Imagen restaurada (con ruido y desenfoque).
    """
    #--- acá va su código aquí ----
    
    #------------------------------ 
    return X_recon 

mostrar en pantalla la imagen reconstruida

In [ ]:
### Solo debe ejecutar esta celda
X_recon = restaurar_imagen(B_data,Ar,k=50)  

plt.figure(figsize=(5,4))
plt.imshow(X_recon, cmap='gray')
plt.axis('off')
plt.title('Reconstrucción completa')
plt.tight_layout()
plt.show()

# 2. K óptimo para la reconstrucción mediante MRE.

Dado un conjunto de datos de imágenes difuminadas y con ruido almacenadas en la carpeta `imgResBlur/` y sus imágenes originales `imgRes/` correspondiente, es posible calcular el $k$ óptimo para la restauración de imágenes con ruido mediante la minimización del MRE,  el cual corresponde a:
$$ 
\Phi_{MRE}(k) = \frac{1}{N} \sum^{N}_{n=1} \mathcal{E}(n,k),
$$
donde $\mathcal{E} (n,k)$ corresponde al error relativo absoluto de la $n$-ésima imágen en comparación con la reconstrucción de la $n$-ésima imagen usando $k$ componentes. $N$ corresponde a la cantidad de imágenes en el dataset.

### Utils:
Función para cargar los datos y variables globales.

In [ ]:
def loadData(directorio='imgRes/', batch=10):
    try:
        archivos = os.listdir(directorio)
        archivos = sorted(archivos)
    except FileNotFoundError:
        print(f"El directorio {directorio} no existe")
        return 0

    i = 0
    data = []
    for imagen in archivos:
        img = Image.open(directorio+imagen)
        data.append(np.array(img))
    
        i += 1
        if i == batch:
            break
    data = np.stack(data)
    return data

#Variables globales:
imgShape = 128
_, Arm = blur_operator(imgShape, imgShape, bl=1)

#Cargar ground Truth
imgsTrue = loadData(directorio='imgRes/', batch=40)

#Cargar imágenes con blur
imgsBlur = loadData(directorio='imgResBlur/', batch=40)

### Pregunta 2.1 (10 pts)
Construya la función que calcule el **error relativo** entre la imagen original $Y$ y su respectiva reconstrucción $X$, es decir debe obtener la siguiente expresión,
$$
    err = \dfrac{\|X-Y\|_{\text{F}}}{\|Y\|_{\text{F}}},
$$
donde $\|\cdot\|_{\text{F}}$ corresponde a la norma matricial de Frobenious.


In [ ]:
def relativo(X, Y):
    """
    input:
    X: ndarray
        Imagen reconstruida
    Y: ndarray
        Imagen original
    output:
    err : float
        Error relativo absoluto 
    """
    #--- acá va su código aquí ----
    
    #------------------------------ 
    return err

El siguiente código obtiene el MRE pero requiere la implementación previa de `restaurar_imagen` y `error_relativo`.

In [ ]:
def getMRE(B, T, Ar, k):
    """
    input:
    B : ndarray
        Lista de imágenes borrosas y ruidosas, 
            donde B[i] es la i-ésima imagen degradada
    T: ndarray
        Lista de imágenes originales, 
            donde T[i] es la i-ésima imagen original
    Ar : ndarray
        Matriz Toeplitz que difumina verticalmente (filas).
    k: int
        k componentes para reconstruir las imágenes.
    output:
    mre : ndarray
        Valor del mre.
    """
    mre = 0
    for n in range(T.shape[0]):      
        X_rest = restaurar_imagen( B[n], Ar, k)
        mre += relativo(X_rest, T[n])
    mre /= T.shape[0]
    return mre

## Pregunta 3
Ejecute la celda

**Nota: Se demora menos de 1 minuto.**

In [ ]:
K = np.logspace(0, 3, 80, dtype=int)
fun_mre = lambda k: getMRE(B=imgsBlur, T=imgsTrue, Ar=Arm, k=k)
fun_mre_vectorized = np.vectorize(fun_mre)
res = fun_mre_vectorized(K)

plt.figure()
plt.loglog(K, res, '.', label='mre')
plt.grid(True)
plt.ylabel('$\\Phi_{MRE}(k)$')
plt.xlabel('k')
plt.legend()
plt.show()

### Pregunta 3.1 (20 pts)
Explique a qué se debe el comportamiento en los extremos del dominio del experimento numérico anterior.

----------------------------------------------------------------------
**Respuesta:** 

----------------------------------------------------------------------

### Pregunta 3.2 (10 pts)
El fundamento de hacer el experimento numérico anterior es obtener una $k$ óptimo y aplicarlo a imágenes degradadas, y que no se tiene acceso a su imagen original. 
En base al gráfico anterior, escoja un valor el $k$ óptimo y **justifique** su elección.

**NOTA: En la siguiente celda debe incluir el valor de $k$ elegido para que pueda ver su resultado.**

--------------------------------------------------------
**Respuesta:** 

--------------------------------------------------------

In [ ]:
#----coloque el valor de k aquí----
k_optimo = 
#----------------------------------------------
print(k_optimo)

In [ ]:
img = Image.open("end_B.jpg").convert("L")
img = img.resize((128,128))

plt.figure(figsize=(10, 8))

plt.subplot(1, 2, 1)
plt.imshow(img, cmap='gray')
plt.title("Image blurry and noisy")
plt.axis('off')

plt.subplot(1, 2, 2)
img_res = restaurar_imagen(img, Arm, k_optimo)
plt.imshow(img_res, cmap='gray')
plt.title("Image noisy and blurry fixed")
plt.axis('off')

plt.tight_layout()
plt.show()